# Install the necessary libraries

In [1]:
from ultralytics import YOLO
import os
import shutil
from codecarbon import OfflineEmissionsTracker
import torch
pathInicial=os.getcwd() 
os.chdir("yolov8")
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.12.1+cu113 _CudaDeviceProperties(name='Quadro RTX 4000', major=7, minor=5, total_memory=7982MB, multi_processor_count=36)


# We see if we have the data in place.

In [2]:
if not os.path.exists("../DatasetMobilityAid/dataYOLOv5"):
    print("You have to follow the instructions in Readme.md for Dataset Mobility Aid and download the data")
else:
    print("Good,¡The data is already in place!")

Good,¡The data is already in place!


# Decide if training with the full train dataset or if reducing dataset

You must select one of the possible methods:

- NONE
- SRS
- DES
- MMS
- NRMD
- RKM
- PHL
- PRD
- FES

You must select a reduction rate, for example if you want a reduction rate of 75%, you have to insert 0.25 in perc, that is to remain only with the 25% of full data.

First, we are going to create a neural network with all the train data and then one with a subset of the 50% of the data.

## Train with full train data

In [3]:
os.chdir(pathInicial)
method = "NONE"
perc=0.25
!python ReductionDatasetMobilityAid_yolov8.py --name {method} --perc {perc}

2024-09-13 06:19:59.152597: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-13 06:19:59.168556: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-13 06:19:59.173328: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
You have not selected any method, so you are going to train with the complete training set.
Number Original Files: 10961
The training set has a size of:  10961


We updated the data yaml, in order to correctly run the training and evaluate the model

In [10]:
os.chdir(pathInicial + "/DatasetMobilityAid/dataYOLOv5")

import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [11]:
%%writetemplate data.yaml
names:
- pedestrian
- wheelchair
- push_wheelchair
- crutches
- walking-frame
nc: 5
test: test/images
train: train/images
val: test/images

#  We train YOLOV8 on our reduced dataset

Descargamos modelo

In [13]:
os.chdir(pathInicial + '/yolov8')
model = YOLO("yolov8m.pt")

Congelamos 10 capas iniciales

In [14]:
freeze = 10
freeze = [f'model.{x}.' for x in range(freeze)]  
for k, v in model.named_parameters(): 
    v.requires_grad = True # train all layers 
    if any(x in k for x in freeze): 
      v.requires_grad = False
    print(k, v.requires_grad)

model.model.0.conv.weight False
model.model.0.bn.weight False
model.model.0.bn.bias False
model.model.1.conv.weight False
model.model.1.bn.weight False
model.model.1.bn.bias False
model.model.2.cv1.conv.weight False
model.model.2.cv1.bn.weight False
model.model.2.cv1.bn.bias False
model.model.2.cv2.conv.weight False
model.model.2.cv2.bn.weight False
model.model.2.cv2.bn.bias False
model.model.2.m.0.cv1.conv.weight False
model.model.2.m.0.cv1.bn.weight False
model.model.2.m.0.cv1.bn.bias False
model.model.2.m.0.cv2.conv.weight False
model.model.2.m.0.cv2.bn.weight False
model.model.2.m.0.cv2.bn.bias False
model.model.2.m.1.cv1.conv.weight False
model.model.2.m.1.cv1.bn.weight False
model.model.2.m.1.cv1.bn.bias False
model.model.2.m.1.cv2.conv.weight False
model.model.2.m.1.cv2.bn.weight False
model.model.2.m.1.cv2.bn.bias False
model.model.3.conv.weight False
model.model.3.bn.weight False
model.model.3.bn.bias False
model.model.4.cv1.conv.weight False
model.model.4.cv1.bn.weight False


Entrenamos el modelo

In [15]:
model.train(data = "../DatasetMobilityAid/dataYOLOv5/data.yaml", batch=16, imgsz=640, epochs=50, name ="train_yolov8_mobilityaid_GAM24_fullTrainSet", seed = 0)

New https://pypi.org/project/ultralytics/8.2.92 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.91 🚀 Python-3.9.19 torch-1.12.1+cu113 CUDA:0 (Quadro RTX 4000, 7982MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=../DatasetMobilityAid/dataYOLOv5/data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train_yolov8_mobilityaid_GAM24_fullTrainSet6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=F

train: Scanning /home/vtoscano/VToscano/GA_M24/DatasetMobilityAid/dataYOLOv5/train/labels... 10961 images, 0 backgrounds


train: New cache created: /home/vtoscano/VToscano/GA_M24/DatasetMobilityAid/dataYOLOv5/train/labels.cache


val: Scanning /home/vtoscano/VToscano/GA_M24/DatasetMobilityAid/dataYOLOv5/test/labels... 6118 images, 27 backgrounds, 0


val: New cache created: /home/vtoscano/VToscano/GA_M24/DatasetMobilityAid/dataYOLOv5/test/labels.cache
Plotting labels to runs/detect/train_yolov8_mobilityaid_GAM24_fullTrainSet6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train_yolov8_mobilityaid_GAM24_fullTrainSet6
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       6118      12990      0.906      0.855      0.932      0.686



5 epochs completed in 0.397 hours.
Optimizer stripped from runs/detect/train_yolov8_mobilityaid_GAM24_fullTrainSet6/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train_yolov8_mobilityaid_GAM24_fullTrainSet6/weights/best.pt, 52.0MB

Validating runs/detect/train_yolov8_mobilityaid_GAM24_fullTrainSet6/weights/best.pt...
Ultralytics YOLOv8.2.91 🚀 Python-3.9.19 torch-1.12.1+cu113 CUDA:0 (Quadro RTX 4000, 7982MiB)
Model summary (fused): 218 layers, 25,842,655 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 192/192 [00:4


                   all       6118      12990      0.906      0.855      0.932      0.686
            pedestrian       3665       6208      0.975      0.626      0.881      0.601
            wheelchair       1993       1993      0.964      0.884      0.942      0.683
       push_wheelchair        782        782      0.979      0.895      0.971      0.761
              crutches       1833       1833      0.697       0.96      0.929      0.704
         walking-frame       2174       2174      0.915       0.91      0.936      0.682
Speed: 0.1ms preprocess, 3.8ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train_yolov8_mobilityaid_GAM24_fullTrainSet6


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f407ed4ef70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

Cogemos el mejor modelo de los entrenados y validamos el modelo en el conjunto de test, no hace falta argumentos, ya que la configuración y el dataset son recordados de manera automática

In [17]:
bestmodel = YOLO("runs/detect/train_yolov8_mobilityaid_GAM24_fullTrainSet/weights/best.pt")

bestmodel.val()

Ultralytics YOLOv8.2.91 🚀 Python-3.9.19 torch-1.12.1+cu113 CUDA:0 (Quadro RTX 4000, 7982MiB)
Model summary (fused): 218 layers, 25,842,655 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /home/vtoscano/VToscano/GA_M24/DatasetMobilityAid/dataYOLOv5/test/labels.cache... 6118 images, 27 backgrou
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 383/383 [01:0


                   all       6118      12990      0.906      0.855      0.931      0.686
            pedestrian       3665       6208      0.975      0.627       0.88      0.601
            wheelchair       1993       1993      0.964      0.884      0.941      0.682
       push_wheelchair        782        782      0.979      0.895      0.971      0.761
              crutches       1833       1833      0.696       0.96      0.929      0.704
         walking-frame       2174       2174      0.915       0.91      0.936      0.682
Speed: 0.1ms preprocess, 8.5ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/val3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f40d4452970>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

Make the inference in the video we have choose for the M24 GA of REXASI-PRO. if we want only to detect the pedestrian and people in wheelchairs: classes = [0 1]

In [18]:
bestmodel.predict("../GAM24_video.mp4",save=True,imgsz=640,conf=0.4, classes = [0, 1])



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/664) /home/vtoscano/VToscano/GA_M24/yolov8/../GAM24_video.mp4: 384x640 3 pedestrians, 17.1ms
video 1/1 (frame 2/664) /home/vtoscano/VToscano/GA_M24/yolov8/../GAM24_video.mp4: 384x640 3 pedestrians, 16.3ms
video 1/1 (frame 3/664) /home/vtoscano/VToscano/GA_M24/yolov8/../GAM24_video.mp4: 384x640 3 pedestrians, 1 wheelchair, 16.3ms
video 1/1 (frame 4/664) /home/vtoscano/VToscano/GA_M24/yolov8/../GAM24_video.mp4: 384x640 2 pedestrians, 1

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'pedestrian', 1: 'wheelchair', 2: 'push_wheelchair', 3: 'crutches', 4: 'walking-frame'}
 obb: None
 orig_img: array([[[37, 36, 38],
         [37, 36, 38],
         [37, 36, 38],
         ...,
         [31, 30, 32],
         [31, 30, 32],
         [31, 30, 32]],
 
        [[37, 36, 38],
         [37, 36, 38],
         [37, 36, 38],
         ...,
         [31, 30, 32],
         [31, 30, 32],
         [31, 30, 32]],
 
        [[37, 36, 38],
         [37, 36, 38],
         [37, 36, 38],
         ...,
         [31, 30, 32],
         [31, 30, 32],
         [31, 30, 32]],
 
        ...,
 
        [[14,  4,  0],
         [14,  4,  0],
         [14,  4,  0],
         ...,
         [39, 21, 26],
         [37, 19, 24],
         [37, 19, 24]],
 
        [[19,  9,  4],
         [19,  9,  4],
         [19,  9,  4],
         ...,
         [38, 20, 25]

## Train with a subset of the 50% of the data

In [ ]:
os.chdir(pathInicial)
method = "SRS"
perc=0.5
!python ReductionDatasetMobilityAid_yolov8.py --name {method} --perc {perc}

In [ ]:
os.chdir(pathInicial + '/yolov8')
model = YOLO("yolov8m.pt")

freeze = 10
freeze = [f'model.{x}.' for x in range(freeze)]  
for k, v in model.named_parameters(): 
    v.requires_grad = True # train all layers 
    if any(x in k for x in freeze): 
      v.requires_grad = False
    print(k, v.requires_grad)

model.train(data = "../DatasetMobilityAid/dataYOLOv5/data.yaml", batch=16, imgsz=640, epochs=50, name ="train_yolov8_mobilityaid_GAM24_subset", seed = 0)

In [ ]:
bestmodel = YOLO("runs/detect/train_yolov8_mobilityaid_GAM24_subset/weights/best.pt")

bestmodel.val()

In [ ]:
bestmodel.predict("../GAM24_video.mp4",save=True,imgsz=640,conf=0.4, classes = [0, 1])